In [100]:
# Dependencies and Setup
import pandas as pd

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_rename = school_data_complete.rename(columns={"student_name":"Student Name", "gender":"Gender", "grade":"Grade", "school_name":"School", "reading_score":"Reading Score", "math_score":"Math Score", "budget":"Budget", "type":"Type"})

school_data_complete_rename.head()

array([1910635, 1884411, 1056600, 3022020,  917500, 1319574, 1081356,
       3124928,  248087,  585858, 1049400, 2547363, 3094650, 1763916,
       1043130], dtype=int64)

In [101]:
#District Summary

#Total Schools
schools_unique = school_data_complete_rename["School"].unique()
total_schools = len(schools_unique)

#Total Students
total_students = len(school_data_complete_rename["Student ID"])

#Total Budget
budgets = school_data_complete_rename["Budget"].unique()
total_budget = budgets.sum()

#Average Math Score
average_math_score = school_data_complete_rename["Math Score"].mean()

#Average Reading Score
average_reading_score = school_data_complete_rename["Reading Score"].mean()

#Overall Pass Rate (Overall Average Score)
overall_pass_rate = float((average_reading_score + average_math_score)) / 2

#Percent students passing math
pass_total_m = 0
for score in school_data_complete_rename["Math Score"]:
    if score >= 70:
        pass_total_m += 1
percent_pass_m = float((pass_total_m) / len(school_data_complete_rename["Math Score"])) * 100

#Percent students passing reading
pass_total_r = 0
for score in school_data_complete_rename["Reading Score"]:
    if score >= 70:
        pass_total_r += 1
percent_pass_r = float((pass_total_r) / len(school_data_complete_rename["Reading Score"])) * 100

#Put data in a dataframe
data_summary_df = pd.DataFrame({"Total Number Schools":[total_schools], "Total Number Students":[total_students], "Average Reading Score":[average_reading_score], "Reading Pass Rate":[percent_pass_r], "Average Math Score":[average_math_score], "Math Pass Rate":[percent_pass_m], "Overall Pass Rate":[overall_pass_rate], "Total Budget":[total_budget]})

#Formatting Data
data_summary_df["Total Number Students"] = data_summary_df["Total Number Students"].map("{:,}".format)
data_summary_df["Average Reading Score"] = data_summary_df["Average Reading Score"].map("{:.2f}".format)
data_summary_df["Reading Pass Rate"] = data_summary_df["Reading Pass Rate"].map("{:.2f}%".format)
data_summary_df["Average Math Score"] = data_summary_df["Average Math Score"].map("{:.2f}".format)
data_summary_df["Math Pass Rate"] = data_summary_df["Math Pass Rate"].map("{:.2f}%".format)
data_summary_df["Overall Pass Rate"] = data_summary_df["Overall Pass Rate"].map("{:.2f}%".format)
data_summary_df["Total Budget"] = data_summary_df["Total Budget"].map("${:,.2f}".format)
data_summary_df.head()

,Total Number Schools,Total Number Students,Average Reading Score,Reading Pass Rate,Average Math Score,Math Pass Rate,Overall Pass Rate,Total Budget
0,15,"39,170",81.88,85.81%,78.99,74.98%,80.43%,"$24,649,428.00"


In [112]:
#School Summary

#Group data by school
data_by_school = school_data_complete_rename.groupby("School")

#School type
school_type = data_by_school["Type"].unique()
data_by_school_df = pd.DataFrame({"Type":school_type})
data_by_school_df.loc[["Bailey High School", "Figueroa High School", "Ford High School", "Hernandez High School", "Huang High School", "Johnson High School", "Rodriguez High School"], "Type"] = "District"
#Number of students
data_by_school_df["Number of Students"] = data_by_school["Student ID"].count().map("{:,}".format)
#School Summary

#Group data by school
data_by_school = school_data_complete_rename.groupby("School")

#School type
school_type = data_by_school["Type"].unique()
data_by_school_df = pd.DataFrame({"Type":school_type})
data_by_school_df.loc[["Bailey High School", "Figueroa High School", "Ford High School", "Hernandez High School", "Huang High School", "Johnson High School", "Rodriguez High School"], "Type"] = "District"
data_by_school_df.loc[["Cabrera High School", "Griffin High School", "Holden High School", "Pena High School", "Shelton High School", "Thomas High School", "Wilson High School", "Wright High School"], "Type"] = "Charter"

#Number of Students per School
data_by_school_df["Number of Students"] = data_by_school["Student ID"].count()

#School Budget
data_by_school_df["Budget"] = budgets
data_by_school_df["Budget"] = data_by_school_df["Budget"]

#Budget Per Student
data_by_school_df["Budget Per Student"] = (data_by_school_df["Budget"] / data_by_school_df["Number of Students"]).map("${:,.2f}".format)

#Formating Number of Students and Budget
data_by_school_df["Number of Students"] = data_by_school_df["Number of Students"].map("{:,}".format)
data_by_school_df["Budget"] = data_by_school_df["Budget"].map("${:,.2f}".format)
data_by_school_df

,Type,Number of Students,Budget,Budget Per Student
School,,,,
Bailey High School,District,"4,976","$1,910,635.00",$383.97
Cabrera High School,Charter,"1,858","$1,884,411.00","$1,014.21"
Figueroa High School,District,"2,949","$1,056,600.00",$358.29
Ford High School,District,"2,739","$3,022,020.00","$1,103.33"
Griffin High School,Charter,"1,468","$917,500.00",$625.00
Hernandez High School,District,"4,635","$1,319,574.00",$284.70
Holden High School,Charter,427,"$1,081,356.00","$2,532.45"
Huang High School,District,"2,917","$3,124,928.00","$1,071.28"
Johnson High School,District,"4,761","$248,087.00",$52.11
